In [3]:
%matplotlib widget
import gym
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import os
import sys
sys.path.insert(1, './agent')
sys.path.insert(1, './price_pred')

import stock_env
from QLAgent import QNAgent
from helper import *

In [4]:
name_file_data = 'AAPL.csv'
name_project = 'AAPL_batch'
env_name = 'StockEnv-v0'
name_model_weights = 'trading_weights'
load_model = False
epoch_to_load = 10000

num_episodes = 1000 # training epoches
render_p = 100 # print frequency
init_balance = 1000 # initial fund agent have

len_obs = 30 # observation length, number of days the agent look back
len_window = 1000 # total manipulation length of dataset
interval = 1 # 
overlap = 20
batch_size = 10
a = [i/10 for i in range(-3,5,1)]
print(tuple(a))
action_list=tuple(a)

(-0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4)


In [5]:
path_models, path_stats = check_directories(name_project, len_obs, len_window)
np.random.seed(42)
tf.random.set_seed(42)
# Read data you want to use for trading
train_data, test_data = get_data(f'./data/{name_file_data}')

In [6]:
# Create an instant
env = gym.make(env_name, train_data=train_data, eval_data=test_data, 
               len_obs=len_obs, len_window=len_window, init_balance=init_balance,
               action_list=action_list)
env.seed(42)
print(f'Observation space: {env.observation_space}')
print(f'Action space: {env.action_space}')

Observation space: Box(-5000.0, 5000.0, (30, 1), float32)
Action space: Discrete(8)


In [7]:
# Create an agent
agent = QNAgent(env, discount_rate=0.5, learning_rate=0.1, epsilon=0.1)

In [8]:
# Load model
if load_model:
    agent.load(path_models + name_model_weights + f'_{epoch_to_load}')
    train_statistics = pd.read_csv(path_stats + 'train.csv')
    test_statistics = pd.read_csv(path_stats + 'test.csv')
    init_ep = epoch_to_load
else:
    init_ep = 0
    train_statistics = pd.DataFrame()
    test_statistics = pd.DataFrame()

In [9]:
worth = []
for ep in range(init_ep, num_episodes):
    get_performance(env, agent, train_data=True, training=True, batch_size=batch_size)
    if (ep % render_p) == 0:
        env.render(ep)
    worth.append(env.net_worth)
    if (ep % interval == 0) and not((load_model==True) and (ep == epoch_to_load)):
        agent.model.save_weights(path_models)

        overlap = overlap
        results_train = np.empty(shape=(0, 3))
        results_test = np.empty(shape=(0, 3))

        size_test = ((len(env.eval_data)-env.len_obs-env.len_window) // overlap)+1
        cagr_train, vol_train = get_performance(env, agent, train_data=True, training=False, batch_size=size_test)
        results_train = np.array([np.tile(ep, size_test), cagr_train, vol_train]).transpose()

        cagr_test, vol_test = get_performance(env, agent, train_data=False, training=False, overlap=overlap, batch_size=size_test)
        results_test = np.array([np.tile(ep, size_test), cagr_test, vol_test]).transpose()

        train_statistics = pd.concat([train_statistics, pd.DataFrame(results_train, columns=['epoch', 'cagr','volatility'])])
        train_statistics.to_csv(path_stats+'train.csv', index=False)
        test_statistics = pd.concat([test_statistics, pd.DataFrame(results_test, columns=['epoch', 'cagr','volatility'])])
        test_statistics.to_csv(path_stats+'test.csv', index=False)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


ep 0 
Profit: 170.41823151%
Std: 1.7707009610940467

ep 1 
Profit: 56.36962137999999%
Std: 1.5073226992840596

ep 2 
Profit: 109.89113871000002%
Std: 1.4181940190549025

ep 3 
Profit: 69.91056831%
Std: 0.37422377075014873

ep 4 
Profit: 78.62118091999997%
Std: 1.0314557882732347


        plot_stocks_trading_performance(results_train, path_imgs + f'train_cagr_vol_ep_{ep}',
                                        color='royalblue', acc_title='Train')
        plot_stocks_trading_performance(results_test, path_imgs + f'test_cagr_vol_ep_{ep}',
                                        color='firebrick', acc_title='Test')

In [10]:
plt.figure(figsize=(8,4))
plt.hist(np.array(worth).flatten(), bins=range(500,2000,50))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
all_worth = np.array(worth).flatten()
min_w = np.min(worth).astype('int')
max_w = np.max(worth).astype('int')
plt.figure(figsize=(8,4))
plt.hist(all_worth, bins=range(min_w,max_w,50))
plt.show()

In [ ]:
precision = 100
threshold = 5000
worth_mean = np.mean(worth,axis=-1)
k = np.split(worth_mean, precision)
k = np.mean(k, axis = -1)
#k = [len(np.where(i > threshold)[0]) for i in worth]
plt.figure(figsize=(8,4))
plt.plot(k,'-')
#plt.boxplot(np.transpose(worth))

1049.890100331017

In [ ]:
a = [0,0,0,0]
np.any(a)